<a href="https://colab.research.google.com/github/gz2214/DSGA1006--Capstone-46/blob/main/codes/mistral/Mistral_7B_Instruct_v0_1_Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import HTML, display
from IPython import get_ipython

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import numpy as np

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [4]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Check if the tokenizer has a padding token, set it to EOS token if not
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Essential Features

In [5]:
device = "cuda" # the device to load the model onto

prompt = """You are a psychiatrist who diagnoses patients using the ICD-11 guidelines. Based on the disorder guidelines provided, please write a diagnosis for the following vignette.
Q1 Generalized Anxiety Disorder

Essential (Required) Features: Marked symptoms of anxiety accompanied by either:
general apprehensiveness that is not restricted to any particular environmental circumstance (i.e., “free-floating anxiety”); or
worry (apprehensive expectation) about untoward events occurring in several different aspects of everyday life (e.g., work, finances, health, family). Anxiety and general apprehensiveness or worry are accompanied by additional symptoms, such as:
Muscle tension or motor restlessness. Sympathetic autonomic overactivity as evidenced by frequent gastrointestinal symptoms such as nausea and/or abdominal distress, heart palpitations, sweating, trembling, shaking, and/or dry mouth. Subjective experience of nervousness, restlessness, or being “on edge”. Difficulties maintaining concentration. Irritability. Sleep disturbances (difficulty falling or staying asleep, or restless, unsatisfying sleep). The symptoms are not transient and persist for at least several months, for more days than not. The symptoms are sufficiently severe to result in significant distress about experiencing persistent anxiety symptoms or result in significant impairment in personal, family, social, educational, occupational, or other important areas of functioning.

Q2 Panic Disorder

Essential (Required) Features: Recurrent unexpected panic attacks that are not restricted to particular stimuli or situations. Panic attacks are discrete episodes of intense fear or apprehension also characterized by the rapid and concurrent onset of several characteristic symptoms. These symptoms may include, but are not limited to, the following:
Palpitations or increased heart rate. Sweating. Trembling. Sensations of shortness of breath. Feelings of choking. Chest pain. Nausea or abdominal distress. Feelings of dizziness or lightheadedness. Chills or hot flushes. Tingling or lack of sensation in extremities (i.e., paresthesias). Depersonalization or derealization. Fear of losing control or going mad. Fear of imminent death
Panic attacks are followed by persistent concern or worry (e.g., for several weeks) about their recurrence or their perceived negative significance (e.g., that the physiological symptoms may be those of a myocardial infarction), or behaviours intended to avoid their recurrence (e.g., only leaving the home with a trusted companion). The symptoms are sufficiently severe to result in significant impairment in personal, family, social, educational, occupational, or other important areas of functioning. Panic attacks can occur in other Anxiety and Fear-Related Disorders as well as other Mental and Behavioural Disorders and therefore the presence of panic attacks is not in itself sufficient to assign a diagnosis of Panic Disorder.

Q3 Agoraphobia

Essential (Required) Features: Marked and excessive fear or anxiety that occurs in, or in anticipation of, multiple situations where escape might be difficult or help might not be available, such as using public transportation, being in crowds, being outside the home alone, in shops, theatres, or standing in line. The individual is consistently fearful or anxious about these situations due to a sense of danger or fear of specific negative outcomes such as panic attacks, symptoms of panic, or other incapacitating (e.g., falling) or embarrassing physical symptoms (e.g., incontinence). The situations are actively avoided, are entered only under specific circumstances (e.g., in the presence of a companion), or else are endured with intense fear or anxiety. The symptoms are not transient, that is, they persist for an extended period of time (e.g., at least several months). The symptoms are sufficiently severe to result in significant distress about experiencing persistent anxiety symptoms or significant impairment in personal, family, social, educational, occupational, or other important areas of functioning.

Q4 Specific Phobia

Essential (Required) Features: Marked and excessive fear or anxiety that consistently occurs when exposed to one or more specific objects or situations (e.g., proximity to certain kinds of animals, heights, closed spaces, sight of blood or injury) and that is out of proportion to the actual danger posed by the specific object or situation. The phobic object or situation is actively avoided or else endured with intense fear or anxiety. A pattern of fear, anxiety, or avoidance related to specific objects or situations is not transient, that is, it persists for an extended period of time (e.g., at least several months). The symptoms are sufficiently severe to result in significant distress about experiencing persistent anxiety symptoms or significant impairment in personal, family, social, educational, occupational, or other important areas of functioning.

Q5 Social Anxiety Disorder

Essential (Required) Features: Marked and excessive fear or anxiety that occurs consistently in one or more social situations such as social interactions (e.g., having a conversation), being observed (e.g., while eating or drinking), or performing in front of others (e.g., giving a speech). The individual is concerned that he or she will act in a way, or show anxiety symptoms, that will be negatively evaluated by others (i.e., be humiliating, embarrassing, lead to rejection, or be offensive). Relevant social situations are consistently avoided or endured with intense fear or anxiety. The symptoms are not transient; that is, they persist for an extended period of time (e.g., at least several months). The symptoms are sufficiently severe to result in significant distress about experiencing persistent anxiety symptoms or result in significant impairment in personal, family, social, educational, occupational, or other important areas of functioning.

Q6 Separation Anxiety Disorder

Essential (Required) Features: Marked and excessive fear or anxiety about separation from those individuals to whom the person is attached (i.e., has a deep affective bond with). In children and youth, attachment figures are typically parents, caregivers, or other family members, whereas in adults, they are most often a romantic partner or children. Manifestations of fear or anxiety related to separation depend on the individual’s developmental level, but may include: Persistent thoughts that harm or some other untoward event (e.g., being kidnapped) will lead to separation. Reluctance or refusal to go to school or work. Recurrent excessive distress (e.g., tantrums, social withdrawal) related to being separated from the attachment figure. Reluctance or refusal to go to sleep without being near the attachment figure. Recurrent nightmares about separation. Physical symptoms such as nausea, vomiting, stomachache, headache, on occasions that involve separation from the attachment figure, such as leaving home to go to school or work. The symptoms are not transient, that is, they persist for an extended period of time (e.g., at least several months).
The symptoms are sufficiently severe to result in significant distress about experiencing persistent anxiety symptoms or significant impairment in personal, family, social, educational, occupational, or other important areas of functioning.

Q7 Selective Mutism

Essential (Required) Features: Consistent selectivity in speaking, such that a child demonstrates adequate language competence in specific social situations, typically at home, but consistently fails to speak in others, typically at school. The duration of the disturbance is at least one month, not limited to the first month of school. The disturbance is not due to a lack of knowledge of, or comfort with, the spoken language demanded in the social situation. Selectivity of speech is sufficiently severe so as to interfere with educational achievement or with social communication or is associated with significant impairment in other important areas of functioning.

Q8 Other Anxiety and Fear-Related Disorder

Essential (Required) Features: The clinical presentation does not satisfy the definitional requirements of any other disorder in the Anxiety and Fear-Related Disorders grouping. The symptoms are not better explained by another Mental and Behavioural Disorder (e.g., a psychotic, mood, or obsessive-compulsive and related disorder). The clinical presentation is judged to be a Mental and Behavioural Disorder that shares primary clinical features with other Anxiety and Fear-Related Disorders (e.g., physiological symptoms of excessive arousal, apprehension, and avoidance behaviour). The symptoms are not developmentally appropriate. The symptoms are in excess of what is normative for the individual’s specific cultural context. The symptoms and behaviours are not explained by another medical disorder that is not classified under Mental and Behavioural Disorders. The symptoms cause significant distress or significant impairment in personal, family, social, educational, occupational or other important areas of functioning.

Q9 Unspecified Anxiety and Fear-Related Disorder

Essential (Required) Features: (No specific diagnostic guidance provided.)

Q10 A different diagnosis

Q11 No diagnosis

Diagnosis example:
Vignette 5B. Vignette 5B will be a case of Social Anxiety Disorder that also displays regular panic attacks in or in anticipation of social evaluative situations. It should be clear that the panic attacks are always situationally cued, and thus are not unexpected. There is no focus of concern on the panic per se, the person’s anxiety is focused on the evaluative nature of the situation, such that people might notice the symptoms of anxiety and judge the person negatively on that basis.
Referral:
JL is a 28-year-old mother of two young children who was referred by her family physician for a psychiatric evaluation due to reported episodes of extreme anxiety that have made it difficult for her to accomplish her daily tasks.

Presenting Symptoms:
During the interview, JL reports that she is happily married and is proud to be caring full time for her daughter and son, aged 2 and 8. She indicates that she works hard to ensure that her family is well cared for, accomplishing her household chores with thoughtfulness and special attention (e.g., buying healthy food and preparing wholesome meals, researching interesting crafts to do with her children, tidying the house every day). JL says that her current difficulties began when her son started primary school two years ago, and her regular routine was disrupted because she had to pick him up at the end of the school day. When the psychiatrist asks JL why this was disruptive, she replies that she often has to wait for her son among the other parents and make “small talk.” She indicates that she “dreads” these exchanges because she feels that the other parents judge what she says as “stupid.” However, when asked, she could not provide a specific example of something that any of the parents said that was judgmental. JL’s son attends a school where parents are expected to volunteer, and JL wants to do this “to be a good mother.”  Since the middle of the school year, JL has acted as a lunch monitor twice a week with several other parents. She describes this activity as “hell on earth” and reports that on a number of occasions she has experienced “the worst anxiety imaginable.” She states that several times, when talking with the other parents, she has experienced a lot of “distressing” symptoms that come on suddenly, including feeling shaky, faint, heart palpitations, and a sensation that she is “outside of her body.”  Now, she has started to feel very anxious that she will have another episode from the time she wakes up on the days she is scheduled to volunteer at the school, and says, “I don’t want to put myself through that.”  On several occasions, she has called the school to say that she did not feel well and could not come in. She thinks that this will make others’ judgments of her even worse. She notes that she only experiences these symptoms of extreme anxiety when interacting with the other parents. JL says that she prefers to stay at home and care for the children. She is happy to spend time with her small group of friends that she has known for many years, but when she attends parties, or other social engagements, she says, “I don’t really enjoy it. I just don’t like having to talk to people I don’t know. I never know what to say and they probably think I am stupid.”

Additional Background Information:
JL reports that she has always been in good health, except for an appendectomy 10 years ago. She does not smoke or drink alcohol and has never used any substances.

CORRECT DIAGNOSIS: ICD-11: Social Anxiety Disorder

Reason why Diagnosis example is correct: Based on the provided vignette and guidelines, JL's case aligns well with the diagnosis of Social Anxiety Disorder (SAD) as per ICD-11 criteria. SAD is characterized by marked and excessive fear or anxiety in social situations, where the individual fears negative evaluation by others. JL exhibits this through her dread of interacting with other parents and her perception that they judge her negatively, despite lacking concrete examples of such judgment. Her anxiety is specifically triggered in social contexts (e.g., school volunteering, parties), leading to avoidance behaviors (e.g., calling in sick) and significant distress. Additionally, her panic attacks, which include symptoms like shakiness, heart palpitations, and depersonalization, occur in anticipation of or during social interactions, further supporting the SAD diagnosis. These symptoms interfere with her daily functioning and are persistent, fulfilling the criteria for SAD. The absence of substance use or other health issues rules out alternative causes, solidifying the appropriateness of the SAD diagnosis for JL.

Vignette:
Referral:
TM is a 16-year-old male student who lives with his mother and younger sister. TM was brought for evaluation at the university psychiatric clinic by his mother because of her concern about TM’s frequent absences from school.

Presenting Symptoms:
During the evaluation, TM does not readily make eye contact with the clinician and remains quiet during the first part of the session. TM’s mother reports TM has always been a “shy but likeable kid.” His avoidance of going to school started at the beginning of the current school year, about 8 months ago, after his family moved and he started attending a new school. On most school mornings, TM begs her to let him stay home. TM’s mother tries to force him to go, but says that “sometimes it is just too much of a battle.” She became seriously concerned when the school notified her that he had missed several school days when she thought he had gone to school. TM is not doing well in school because, in addition to his many absences, when the teacher calls on him, he says “I just freeze. Even if I know the answer, I can’t say it. I just turn all red and look like an idiot.” TM admits that he has always had a hard time talking to people he doesn’t know well. He had a few friends at his former school because “I was at the same school my whole life and we all knew each other.” He is having great difficulty making friends at his new school, because he is worried that the other kids think he is “a total loser.” TM thinks about this all the time and says he has trouble sleeping because of it. He would like to have friends and be popular, and wishes he could be more relaxed and confident enough to interact with new people.
TM’s mother mentions that TM spends most evenings and weekends reading or playing video games in his room, activities that he says he enjoys thoroughly. TM used to play soccer at his old school, and was considered a good player. However, he has been unable to introduce himself to the coach at the new school, and try out for the team, because he would have to do this in front of the other players. He is worried that he would do a bad job during the try-out because of his nervousness, and the other players would look down on him for not being good enough. TM’s mother reports that she was initially concerned that TM was being bullied at school, but after speaking with his teachers and parents of classmates, she found no indication that he was being harassed. TM eventually responds, “No, it’s not them. It’s me.”

Additional Backgroud Information:
 TM’s parents are divorced and his father lives in another city, but they speak over the phone several times a week. After his mother is asked to leave the room, TM reports that he does not use drugs, but does admit that he has drunk alcohol on several occasions and once smoked a cigarette. At a recent check-up, TM’s family doctor found him to be in good health.


Given this Vignette, what diagnosis would you make? Answer in steps described:
Step 1. Describe the patient's presenting symptoms, medical history, family history, and any relevant lifestyle factors.
Step 2. Based on the patient's presenting symptoms and additional background information, what are the possible differential diagnoses? (Choose the top 3).
Step 3. Why did you choose these diagnoses?
Step 4. Which diagnosis matches the patient's symptoms and history the most?
Step 5. What is the final diagnosis for the patient, and why? If it is a generalized anxiety disorder, also provide the next likely diagnosis.
"""

messages = [
    {"role": "user", "content": prompt}
]

max_length = 512

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

model_inputs = encodeds.to(device);

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
generated_text = decoded[0]
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] You are a psychiatrist who diagnoses patients using the ICD-11 guidelines. Based on the disorder guidelines provided, please write a diagnosis for the following vignette.
Q1 Generalized Anxiety Disorder

Essential (Required) Features: Marked symptoms of anxiety accompanied by either:
general apprehensiveness that is not restricted to any particular environmental circumstance (i.e., “free-floating anxiety”); or
worry (apprehensive expectation) about untoward events occurring in several different aspects of everyday life (e.g., work, finances, health, family). Anxiety and general apprehensiveness or worry are accompanied by additional symptoms, such as:
Muscle tension or motor restlessness. Sympathetic autonomic overactivity as evidenced by frequent gastrointestinal symptoms such as nausea and/or abdominal distress, heart palpitations, sweating, trembling, shaking, and/or dry mouth. Subjective experience of nervousness, restlessness, or being “on edge”. Difficulties maintainin

In [6]:
import locale
#locale.getpreferredencoding = lambda: "UTF-8"

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [9]:
!pip install evaluate

In [7]:
from torch.nn import CrossEntropyLoss
import evaluate

# Tokenize the text (add special tokens for beginning and end of the sentence)
inputs = tokenizer(generated_text, return_tensors="pt", truncation=True, padding=True)
inputs = inputs.to('cuda')

# Get model outputs
with torch.no_grad():
    outputs = model(**inputs, labels=inputs['input_ids'])

# Calculate cross-entropy loss
loss_fct = CrossEntropyLoss()
loss = loss_fct(outputs.logits.view(-1, model.config.vocab_size), inputs['input_ids'].view(-1))

# Compute perplexity
perplexity = torch.exp(loss).item()

print(f"Perplexity: {perplexity}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Perplexity: 759605.1875


In [10]:
import evaluate

# Load the perplexity metric
perplexity = evaluate.load("perplexity", module_type="metric")

# Compute the perplexity for your generated text
# You can adjust 'model_id' based on the model you want to use for calculating perplexity
results = perplexity.compute(model_id=model_name, data=[generated_text])

# Output the results
print(f"Mean Perplexity: {results['mean_perplexity']}")
for i, ppl in enumerate(results['perplexities']):
    print(f"Text {i+1} Perplexity: {ppl}")

ValueError: ignored

Essential Features + Additional features

Essential Features + Additional Information + Boundary

Summarized some long paragraphs

In [ ]:
device = "cuda" # the device to load the model onto

prompt = """You are a psychiatrist who diagnoses patients using the ICD-11 guidelines. Based on the disorder guidelines provided, please write a diagnosis for the following vignette.
Q1 Generalized Anxiety Disorder:
Essential Features: The condition described involves pronounced anxiety symptoms, manifesting as persistent, non-specific apprehension or worry about various aspects of daily life such as work, finances, and health. These feelings of anxiety are accompanied by physical and psychological symptoms, including muscle tension, restlessness, sympathetic overactivity indicated by gastrointestinal issues, palpitations, and sweating, as well as feelings of nervousness and being on edge. Additional symptoms include concentration difficulties, irritability, and sleep disturbances. This condition is not short-lived but persists for several months on most days, causing significant distress or impairment in important areas of functioning like personal, family, social, or occupational life.
Additional Features: Some individuals with Generalized Anxiety Disorder may only report chronic somatic anxiety and a feeling of nonspecific dread without being able to articulate specific worry content. Behavioural changes such as avoidance, frequent need for reassurance (especially in children), and procrastination may be seen. These behaviours typically represent an effort to reduce apprehension or prevent untoward events from occurring.
Boundary with Normality: Anxiety and worry are typical responses to stress, often aiding in problem-solving, focusing attention, and enhancing alertness. When regulated, these emotions don't usually disrupt daily functioning or cause significant distress. However, in Generalized Anxiety Disorder (GAD), anxiety and worry become excessive, persistent, and intensely disruptive. Importantly, intense anxiety experienced in highly stressful situations, such as living in a war zone, may be a natural and appropriate response to the environment. Such instances of anxiety and worry, when confined to extreme circumstances, shouldn't be mistaken for symptoms of GAD.

Q2 Panic Disorder:
Essential Features: Panic attacks are characterized by recurrent, unexpected episodes of intense fear or apprehension, not linked to specific stimuli or situations. They manifest rapidly with various symptoms like palpitations, sweating, trembling, breathlessness, feelings of choking, chest pain, nausea, dizziness, temperature fluctuations, tingling sensations, depersonalization, and intense fears of losing control or imminent death. These episodes lead to ongoing worry about their recurrence, impacting significant life areas, and are accompanied by avoidance behaviors, like only venturing out with a trusted companion. Although panic attacks can occur in various mental and behavioral disorders, their presence alone doesn't necessarily indicate a Panic Disorder diagnosis. Their severe impact on personal, family, social, educational, and occupational functioning is notable.
Additional Features: Panic attacks typically last only minutes but can vary greatly in frequency and severity, ranging from several times a day to a few times per month. Individuals with Panic Disorder often experience limited-symptom attacks, which are similar to full panic attacks but with fewer symptoms. To be diagnosed with Panic Disorder, one must have a history of recurrent panic attacks that meet specific criteria. Agoraphobia can be diagnosed without Panic Disorder if the individual does not have a history of recurrent full-symptom panic attacks but meets all criteria for Agoraphobia. Some with Panic Disorder also suffer from nocturnal panic attacks, waking up in a state of panic. Despite the variability in symptoms, severity, and avoidance behaviors, Panic Disorder is highly impairing, leading individuals to frequently visit emergency units and undergo numerous medical tests, often with negative results.
Boundary with normality: Panic attacks are characterized by their abrupt onset, rapid escalation, intense severity, and unpredictability, which sets them apart from the typical anxiety experienced in everyday situations, like during stressful life events. Panic Disorder, in particular, is distinguished by the frequent recurrence of such attacks, along with persistent changes in behavior, such as avoidance, and a significant impairment in daily functioning. However, not every instance of a panic attack indicates a disorder; an isolated occurrence without recurrence does not necessarily warrant a diagnosis.

Q3 Agoraphobia:
Essential Features: The condition described involves a persistent, intense fear or anxiety triggered by multiple situations where escape could be challenging or help unavailable, such as using public transportation, being in crowds, or being alone outside. This fear often centers around potential negative outcomes like panic attacks or embarrassing symptoms, leading to avoidance of these situations, reliance on specific conditions like having a companion, or enduring them with extreme discomfort. The enduring nature of these symptoms, lasting for several months or more, significantly impacts the individual's personal, social, and occupational functioning, causing considerable distress and impairment in various aspects of life.
Additional Features: Agoraphobia involves a fear of experiencing panic-like symptoms, such as palpitations and dizziness, or distressing events like vomiting. To cope, individuals with Agoraphobia adopt behavioral strategies like requiring a companion, visiting places at specific times, or carrying emergency medications. These strategies, which can change over time and vary by situation, aim to manage anxiety but may involve enduring significant distress or relying on different safety behaviors. The severity and pattern of symptoms, as well as the extent of avoidance, differ among individuals, leading to varying degrees of impairment. In severe cases, Agoraphobia can lead to individuals becoming completely housebound, negatively impacting their employment, access to healthcare, and personal relationships.
Boundary with normality: Individuals may exhibit transient avoidance behaviours in the context of normal development or in periods of increased stress. These behaviours are differentiated from Agoraphobia because they are limited in duration and do not lead to significant impact on functioning.

Q4 Specific Phobia:
Essential Features:
The condition described is characterized by a marked and excessive fear or anxiety that arises consistently in response to certain objects or situations, such as proximity to specific animals, heights, closed spaces, or seeing blood or injury. This reaction is disproportionate to the actual threat posed. Individuals with this condition actively avoid the feared object or situation or endure it with intense fear or anxiety. The pattern of fear, anxiety, or avoidance is persistent, lasting at least several months, and not just a transient response. The severity of these symptoms causes significant distress, manifesting in persistent anxiety or notably impairs functioning in key areas of life like personal, family, social, educational, and occupational domains.
Additional Features: Specific Phobia is a psychiatric condition characterized by intense fears of a diverse range of stimuli, including animals, heights, enclosed spaces, blood, injury, flying, driving, storms, darkness, and medical or dental procedures. Individuals with this disorder exhibit varying responses, such as disgust, anticipation of danger, and physical symptoms like fainting—particularly in response to blood or injury. It's notable that a single diagnosis encompasses multiple fears, and a unique response observed in some phobic situations is the vasovagal response, leading to fainting. The development of these phobias can stem from direct negative experiences or vicarious learning, though such experiences are not necessary for the disorder's onset. Some individuals may not perceive their fear as excessive; therefore, clinicians must assess whether the fear or anxiety is disproportionate to the actual risk, considering cultural norms and specific environmental conditions.
Boundary with normality: In children and adolescents, some fears may be part of normal development (e.g., a young child who is afraid of dogs). Specific Phobia should only be diagnosed if the fear or anxiety is excessive in comparison to that of other individuals at a similar developmental level.

Q5 Social Anxiety Disorder:
Essential Features: The condition described here involves a persistent and intense fear or anxiety in social situations such as conversations, being watched, or public performance. This anxiety stems from the worry of acting in a manner that may lead to negative judgment, embarrassment, or rejection. Consequently, the individual either avoids these situations or endures them with significant distress. These symptoms are not temporary and last for several months or more, causing considerable distress or impairment in various aspects of life including personal, social, and occupational functioning.
Additional Features:
Social Anxiety Disorder (SAD) typically presents with physical symptoms like blushing, sweating, or trembling, which may overshadow the patient's underlying fear of negative evaluation. This condition often co-occurs with other anxiety and depressive disorders. A significant concern for SAD patients is the potential development of Substance Use Disorders, which can arise as a coping mechanism to alleviate social anxiety. Diagnosing SAD requires a nuanced understanding of the patient's perspective, as they may not view their fear or anxiety as excessive. Clinicians must evaluate the appropriateness of the patient's anxiety considering the social context, cultural norms, and specific factors like bullying, which may validate their response.
Boundary with normality: Social Anxiety Disorder can be differentiated from normal developmental fears (e.g., a child’s reluctance to interact with unfamiliar people in novel situations) by fear and anxiety reactions that are typically excessive, interfere with functioning, and persist over time (e.g., lasting more than several months).

Q6 Separation Anxiety Disorder:
Essential Features: Separation anxiety disorder is characterized by intense and persistent fear or anxiety about being separated from attachment figures, such as parents, caregivers, or a romantic partner. This condition can manifest differently based on an individual's age and developmental level. Symptoms include constant worry about potential harm leading to separation, avoidance of school or work, extreme distress when apart from the attachment figure, resistance to sleeping alone, nightmares about separation, and physical symptoms like nausea or headaches in situations of separation. These symptoms are not short-lived; they continue for several months or more, causing significant distress or impairment in various aspects of the individual's life, including personal, social, and occupational functioning.
Additional Features: Separation Anxiety Disorder often coexists with other mental and behavioral disorders, displaying varying patterns in children and adults. In children, it commonly pairs with Generalized Anxiety Disorder and Specific Phobia, whereas adults with this disorder frequently also suffer from Mood and Anxiety Disorders, Posttraumatic Stress Disorder, and Personality Disorders. Interestingly, many adults with Separation Anxiety Disorder don't remember experiencing it during childhood. This disorder's childhood manifestation is often linked to a parenting style that hampers the development of autonomy and self-reliance, crucial for a person's cultural context, by overly restricting independence in daily activities.
Boundary with normality: Many situations involving separation are associated with other potential stressors or are normal sources of anxiety (e.g., leaving home to start a job or attend university in a new city). Separation Anxiety Disorder is differentiated based on the focus of the anxiety being on separation from a key attachment figure rather than on other aspects of adjusting to novel circumstances.

Q7 Selective Mutism:
Essential Features: Consistent selectivity in speaking, such that a child demonstrates adequate language competence in specific social situations, typically at home, but consistently fails to speak in others, typically at school. The duration of the disturbance is at least one month, not limited to the first month of school. The disturbance is not due to a lack of knowledge of, or comfort with, the spoken language demanded in the social situation. Selectivity of speech is sufficiently severe so as to interfere with educational achievement or with social communication or is associated with significant impairment in other important areas of functioning.
Additional Features: Selective Mutism is a complex condition that affects expressive language in children, who may not speak during assessments but can engage in receptive language tasks. Caregiver observations are vital as children with this disorder often speak only in specific settings. It's commonly associated with Social Anxiety Disorder, characterized by intense anxiety in social situations and fear of speech evaluation. Symptoms usually appear before age 5 and can include subtle language impairments and oppositional behavior in stressful speaking scenarios. This disorder often coexists with other Anxiety Disorders and has significant implications for academic and social functioning, leading to school challenges, social difficulties, and an increased risk of bullying. Behavioral traits like inhibition and negative affectivity can exacerbate social anxiety, withdrawal, and avoidance behaviors in these children.
Boundary with normality: Transient reluctance to speak at the time of first starting school is a common occurrence.  Selective Mutism should only be diagnosed if symptoms persist beyond the first month of schooling. Immigrant children who are unfamiliar with or uncomfortable in the official language of their new host country may, for a discrete period of time, refuse to speak to strangers in their new environment. This may also occur with children from linguistic minorities. Selective Mutism should not be diagnosed in such cases.

Q8 Other Anxiety and Fear-Related Disorder:
Essential Features: The clinical presentation does not satisfy the definitional requirements of any other disorder in the Anxiety and Fear-Related Disorders grouping. The symptoms are not better explained by another Mental and Behavioural Disorder (e.g., a psychotic, mood, or obsessive-compulsive and related disorder). The clinical presentation is judged to be a Mental and Behavioural Disorder that shares primary clinical features with other Anxiety and Fear-Related Disorders (e.g., physiological symptoms of excessive arousal, apprehension, and avoidance behaviour). The symptoms are not developmentally appropriate. The symptoms are in excess of what is normative for the individual’s specific cultural context. The symptoms and behaviours are not explained by another medical disorder that is not classified under Mental and Behavioural Disorders. The symptoms cause significant distress or significant impairment in personal, family, social, educational, occupational or other important areas of functioning.

Q9 Unspecified Anxiety and Fear-Related Disorder: Essential Features: (No specific diagnostic guidance provided.)

Q10 A different diagnosis

Q11 No diagnosis

Diagnosis example:
Vignette 5B. Vignette 5B will be a case of Social Anxiety Disorder that also displays regular panic attacks in or in anticipation of social evaluative situations. It should be clear that the panic attacks are always situationally cued, and thus are not unexpected. There is no focus of concern on the panic per se, the person’s anxiety is focused on the evaluative nature of the situation, such that people might notice the symptoms of anxiety and judge the person negatively on that basis.
Referral:
JL is a 28-year-old mother of two young children who was referred by her family physician for a psychiatric evaluation due to reported episodes of extreme anxiety that have made it difficult for her to accomplish her daily tasks.

Presenting Symptoms:
During the interview, JL reports that she is happily married and is proud to be caring full time for her daughter and son, aged 2 and 8. She indicates that she works hard to ensure that her family is well cared for, accomplishing her household chores with thoughtfulness and special attention (e.g., buying healthy food and preparing wholesome meals, researching interesting crafts to do with her children, tidying the house every day). JL says that her current difficulties began when her son started primary school two years ago, and her regular routine was disrupted because she had to pick him up at the end of the school day. When the psychiatrist asks JL why this was disruptive, she replies that she often has to wait for her son among the other parents and make “small talk.” She indicates that she “dreads” these exchanges because she feels that the other parents judge what she says as “stupid.” However, when asked, she could not provide a specific example of something that any of the parents said that was judgmental. JL’s son attends a school where parents are expected to volunteer, and JL wants to do this “to be a good mother.”  Since the middle of the school year, JL has acted as a lunch monitor twice a week with several other parents. She describes this activity as “hell on earth” and reports that on a number of occasions she has experienced “the worst anxiety imaginable.” She states that several times, when talking with the other parents, she has experienced a lot of “distressing” symptoms that come on suddenly, including feeling shaky, faint, heart palpitations, and a sensation that she is “outside of her body.”  Now, she has started to feel very anxious that she will have another episode from the time she wakes up on the days she is scheduled to volunteer at the school, and says, “I don’t want to put myself through that.”  On several occasions, she has called the school to say that she did not feel well and could not come in. She thinks that this will make others’ judgments of her even worse. She notes that she only experiences these symptoms of extreme anxiety when interacting with the other parents. JL says that she prefers to stay at home and care for the children. She is happy to spend time with her small group of friends that she has known for many years, but when she attends parties, or other social engagements, she says, “I don’t really enjoy it. I just don’t like having to talk to people I don’t know. I never know what to say and they probably think I am stupid.”

Additional Background Information:
JL reports that she has always been in good health, except for an appendectomy 10 years ago. She does not smoke or drink alcohol and has never used any substances.

CORRECT DIAGNOSIS: ICD-11: Social Anxiety Disorder

Reason why Diagnosis example is correct: Based on the provided vignette and guidelines, JL's case aligns well with the diagnosis of Social Anxiety Disorder (SAD) as per ICD-11 criteria. SAD is characterized by marked and excessive fear or anxiety in social situations, where the individual fears negative evaluation by others. JL exhibits this through her dread of interacting with other parents and her perception that they judge her negatively, despite lacking concrete examples of such judgment. Her anxiety is specifically triggered in social contexts (e.g., school volunteering, parties), leading to avoidance behaviors (e.g., calling in sick) and significant distress. Additionally, her panic attacks, which include symptoms like shakiness, heart palpitations, and depersonalization, occur in anticipation of or during social interactions, further supporting the SAD diagnosis. These symptoms interfere with her daily functioning and are persistent, fulfilling the criteria for SAD. The absence of substance use or other health issues rules out alternative causes, solidifying the appropriateness of the SAD diagnosis for JL.

Vignette:
Referral:
TM is a 16-year-old male student who lives with his mother and younger sister. TM was brought for evaluation at the university psychiatric clinic by his mother because of her concern about TM’s frequent absences from school.

Presenting Symptoms:
During the evaluation, TM does not readily make eye contact with the clinician and remains quiet during the first part of the session. TM’s mother reports TM has always been a “shy but likeable kid.” His avoidance of going to school started at the beginning of the current school year, about 8 months ago, after his family moved and he started attending a new school. On most school mornings, TM begs her to let him stay home. TM’s mother tries to force him to go, but says that “sometimes it is just too much of a battle.” She became seriously concerned when the school notified her that he had missed several school days when she thought he had gone to school. TM is not doing well in school because, in addition to his many absences, when the teacher calls on him, he says “I just freeze. Even if I know the answer, I can’t say it. I just turn all red and look like an idiot.” TM admits that he has always had a hard time talking to people he doesn’t know well. He had a few friends at his former school because “I was at the same school my whole life and we all knew each other.” He is having great difficulty making friends at his new school, because he is worried that the other kids think he is “a total loser.” TM thinks about this all the time and says he has trouble sleeping because of it. He would like to have friends and be popular, and wishes he could be more relaxed and confident enough to interact with new people.
TM’s mother mentions that TM spends most evenings and weekends reading or playing video games in his room, activities that he says he enjoys thoroughly. TM used to play soccer at his old school, and was considered a good player. However, he has been unable to introduce himself to the coach at the new school, and try out for the team, because he would have to do this in front of the other players. He is worried that he would do a bad job during the try-out because of his nervousness, and the other players would look down on him for not being good enough. TM’s mother reports that she was initially concerned that TM was being bullied at school, but after speaking with his teachers and parents of classmates, she found no indication that he was being harassed. TM eventually responds, “No, it’s not them. It’s me.”

Additional Backgroud Information:
 TM’s parents are divorced and his father lives in another city, but they speak over the phone several times a week. After his mother is asked to leave the room, TM reports that he does not use drugs, but does admit that he has drunk alcohol on several occasions and once smoked a cigarette. At a recent check-up, TM’s family doctor found him to be in good health.

Given this Vignette, what diagnosis would you make?
"""

messages = [
    {"role": "user", "content": prompt}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: ignored